In [146]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

using LinearAlgebra
using Printf
import ForwardDiff as FD

  Activating environment at `~/SSD/Code/TinyMPC/julia/altro_simple/Project.toml`


In [154]:
# -------------------THIS IS ALL ALTRO-------------------------------
# This enables goal equality constraints, state and input ineq constraints.

function stage_cost(p::NamedTuple,x,u,k)
    dx = x - p.Xref[k]
    du = u - p.Uref[k]
    return 0.5*dx'*p.Q*dx + 0.5*du'*p.R*du
end
function term_cost(p::NamedTuple,x)
    dx = x - p.Xref[p.N]
    return 0.5*dx'*p.Qf*dx
end
function stage_cost_expansion(p::NamedTuple,x,u,k)
    dx = x - p.Xref[k]
    du = u - p.Uref[k]
    return p.Q, p.Q*dx, p.R, p.R*du  # Hessian and gradient
end
function term_cost_expansion(p::NamedTuple,x)
    dx = x - p.Xref[p.N]
    return p.Qf, p.Qf*dx
end
function backward_pass!(params,X,U,P,p,d,K,reg,μ,μx,ρ,λ)
    """iLQR backward pass with AL. This !function update its parameters
    """
    println("solve riccati")
    # backwards pass for Altro
    # P - vector of cost to go quadratic terms (matrices)
    # p - vector of cost to go linear terms (vectors)
    # K - vector of feedback gain matrices (matrices)
    # d - vector of feedforward controls (vectors)

    N = params.N
    ΔJ = 0.0    # expected cost reduction

    # terminal cost expansion
    P[N] = params.Qf 
    p[N] = Qf*X[N]
    # add AL terms for the state constraint at the final time step
#     hxv = ineq_con_x(params,X[N])  # h(x) violation  
#     mask = eval_mask(μx[N],hxv)
#     ∇hx = ineq_con_x_jac(params,X[N])

#     # add these into the cost-to-go p and P
#     p[N]  += ∇hx'*(μx[N] + ρ*(mask * hxv))  # multiplier term (1st)
#     P[N]  += ρ*∇hx'*mask*∇hx                # penalty term (2nd)

#     # add AL terms for goal constraint 
#     hxv = X[N] - params.Xref[N]
#     ∇hx = diagm(ones(params.nx))

#     # add these into the CTG p and P (equality active)
#     p[N]  += ∇hx'*(λ + ρ*hxv)   
#     P[N]  += ρ*∇hx'∇hx

    # iterate from N-1 to 1 backwards
    for k = (N-1):(-1):1

        # dynamics jacobians (linearization about previous iterate)
        A = FD.jacobian(_x -> discrete_dynamics(params,_x,U[k],k),X[k])
        B = FD.jacobian(_u -> discrete_dynamics(params,X[k],_u,k),U[k])

        # one-step cost expansion: Q, Q*dx, R, R*du
        Sx = A' * (p[k+1]) + params.Q*X[k]
        Su = B' * (p[k+1]) + params.R*U[k]
        Sxx = params.Q + A'*(P[k+1])*A
        Suu = params.R + B'*(P[k+1] + reg*I)*B                                     
        Sux = B'*(P[k+1])*A  

        # control constraints
        huv = ineq_con_u(params,U[k])  # calculate h(u) constraint
        mask = eval_mask(μ[k],huv)  # choose active
        ∇hu = ineq_con_u_jac(params,U[k])
        Su  += ∇hu'*(μ[k] + ρ*(mask * huv)) # add to cost
        Suu += ρ*∇hu'*mask*∇hu

        # # state constraints
        # hxv = ineq_con_x(params,X[k])
        # mask = eval_mask(μx[k],hxv)
        # ∇hx = ineq_con_x_jac(params,X[k])
        # Jx  += ∇hx'*(μx[k] + ρ*(mask * hxv))
        # Jxx += ρ*∇hx'*mask*∇hx

        # Calculate Gains
        F = cholesky(Symmetric(Suu))
        d[k] = F\Su
        K[k] = F\Sux

        # Cost-to-go Recurrence (PSD stabilizing version, last term)
        # or P[k] = Gxx + K'GuuK - GxuK - K'Gux
        # P[k] = Jxx + K[k]'*Juu*K[k] + (A-B*K[k])'*P[k+1]*(A-B*K[k])
        # p[k] = Jx - K[k]'*Ju + K[k]'*Juu*d[k] + (A - B*K[k])'*(p[k+1] - P[k+1]*B*d[k])
        P[k] = Sxx + K[k]'*Suu*K[k] - K[k]'*Sux - Sux'*K[k]
        p[k] = Sx + K[k]'*Suu*d[k] - K[k]'*Su - Sux'*d[k]
        ΔJ += Su'*d[k]
    end

    return ΔJ  # but also change params
end
function backward_pass1!(params,X,U,P,p,d,K,reg,μ,μx,ρ,λ)
    """iLQR backward pass with AL. This !function update its parameters
    """
    println("solve riccati")
    # backwards pass for Altro
    # P - vector of cost to go quadratic terms (matrices)
    # p - vector of cost to go linear terms (vectors)
    # K - vector of feedback gain matrices (matrices)
    # d - vector of feedforward controls (vectors)

    N = params.N
    ΔJ = 0.0    # expected cost reduction

    # terminal cost expansion
    P[N], p[N] = term_cost_expansion(params,X[N])   # Vxx, Vx

    # add AL terms for the state constraint at the final time step
#     hxv = ineq_con_x(params,X[N])  # h(x) violation  
#     mask = eval_mask(μx[N],hxv)
#     ∇hx = ineq_con_x_jac(params,X[N])

#     # add these into the cost-to-go p and P
#     p[N]  += ∇hx'*(μx[N] + ρ*(mask * hxv))  # multiplier term (1st)
#     P[N]  += ρ*∇hx'*mask*∇hx                # penalty term (2nd)

#     # add AL terms for goal constraint 
#     hxv = X[N] - params.Xref[N]
#     ∇hx = diagm(ones(params.nx))

#     # add these into the CTG p and P (equality active)
#     p[N]  += ∇hx'*(λ + ρ*hxv)   
#     P[N]  += ρ*∇hx'∇hx

    # iterate from N-1 to 1 backwards
    for k = (N-1):(-1):1

        # dynamics jacobians (linearization about previous iterate)
        A = FD.jacobian(_x -> discrete_dynamics(params,_x,U[k],k),X[k])
        B = FD.jacobian(_u -> discrete_dynamics(params,X[k],_u,k),U[k])

        # one-step cost expansion: Q, Q*dx, R, R*du
        Jxx,Jx,Juu,Ju = stage_cost_expansion(params,X[k],U[k],k)  # or ℓ

        # control constraints
        huv = ineq_con_u(params,U[k])  # calculate h(u) constraint
        mask = eval_mask(μ[k],huv)  # choose active
        ∇hu = ineq_con_u_jac(params,U[k])
        Ju  += ∇hu'*(μ[k] + ρ*(mask * huv)) # add to cost
        Juu += ρ*∇hu'*mask*∇hu

        # # state constraints
        # hxv = ineq_con_x(params,X[k])
        # mask = eval_mask(μx[k],hxv)
        # ∇hx = ineq_con_x_jac(params,X[k])
        # Jx  += ∇hx'*(μx[k] + ρ*(mask * hxv))
        # Jxx += ρ*∇hx'*mask*∇hx

        # Q expansion
        gx = Jx + A'*p[k+1] 
        gu = Ju + B'*p[k+1]

        # not regularized
        # Gxx = Jxx + A'*P[k+1]*A
        # Guu = Juu + B'*P[k+1]*B
        # Gux = B'*P[k+1]*A

        # regularized
        Gxx = Jxx + A'*(P[k+1] + 0*reg*I)*A
        Guu = Juu + B'*(P[k+1] + 0*reg*I)*B
        Gux = B'*(P[k+1] + 0*reg*I)*A

        # Calculate Gains
        F = cholesky(Symmetric(Guu))
        d[k] = F\gu
        K[k] = F\Gux

        # Cost-to-go Recurrence (PSD stabilizing version, last term)
        # or P[k] = Gxx + K'GuuK - GxuK - K'Gux
        # P[k] = Jxx + K[k]'*Juu*K[k] + (A-B*K[k])'*P[k+1]*(A-B*K[k])
        # p[k] = Jx - K[k]'*Ju + K[k]'*Juu*d[k] + (A - B*K[k])'*(p[k+1] - P[k+1]*B*d[k])
        P[k] = Gxx + K[k]'*Guu*K[k] - K[k]'*Gux - Gux'*K[k]
        p[k] = gx + K[k]'*Guu*d[k] - K[k]'*gu - Gux'*d[k]
        ΔJ += gu'*d[k]
    end

    return ΔJ  # but also change params
end
function trajectory_AL_cost(params,X,U,μ,μx,ρ,λ)
    """Calcuate total cost (AL) for the whole trajectory
    """
    N = params.N
    J = 0.0
    for k = 1:N-1
        J += stage_cost(params,X[k],U[k],k)

        # AL terms for ineq_con_u
        huv = ineq_con_u(params,U[k])
        mask = eval_mask(μ[k],huv)
        J += dot(μ[k],huv) + 0.5*ρ*huv'*mask*huv

        # AL terms for ineq_con_x
        hxv = ineq_con_x(params,X[k])
        mask = eval_mask(μx[k],hxv)
        J += dot(μx[k],hxv) + 0.5*ρ*hxv'*mask*hxv
    end

    # AL terms for state constraint at last time step
    # J += term_cost(params,X[N])
    # hxv = ineq_con_x(params,X[params.N])
    # mask = eval_mask(μx[params.N],hxv)
    # J += dot(μx[params.N],hxv) + 0.5*ρ*hxv'*mask*hxv

    # AL terms for goal constraint
    # hxv = X[N] - params.Xref[N]
    # J += dot(λ,hxv) + 0.5*ρ*hxv'*hxv
    return J
end
function forward_pass!(params,X,U,K,d,ΔJ,Xn,Un,μ,μx,ρ,λ; 
                        max_linesearch_iters = 20)
    """iLQR forward pass and line-search
    This !function update its parameters
    """
    N = params.N
    α = 1.0
    # previous cost
    J = trajectory_AL_cost(params,X,U,μ,μx,ρ,λ)
    for i = 1:1
    println("line search")
        # Forward Rollout
        for k = 1:(N-1)
            Un[k] = U[k] - α*d[k] - K[k]*(Xn[k]-X[k])  
            Xn[k+1] = discrete_dynamics(params,Xn[k],Un[k],k)
        end
        # Jn = trajectory_AL_cost(params,Xn,Un,μ,μx,ρ,λ)  # new cost

        # backtracking linesearch (should use Armijo?)
        # if Jn < J
        #     # println("line search - better found")
            X .= Xn
            U .= Un
        #     return Jn, α
        # else
        #     α *= 0.5
        # end
    end

    # @warn "forward pass failed, adding regularization"
    # α = 0.0
    return J, α
end
function update_reg(reg,reg_min,reg_max,α)  
    # Update regularization parameter
    if α == 0.0  # if forward pass fails, enhance regularization
        if reg == reg_max
            error("reached max reg")
        end
        return min(reg_max,reg*10)
    end
    if α == 1.0  # if forward pass is perfect, lower regularization
        return max(reg_min,reg/10)
    end
    return reg
end
function calc_max_d(d)  
    # The feedforward gains go to zero. We compute the
    # average maximum of the normalized gains
    # Just calc the max absolute naively.
    dm = 0.0
    for i = 1:length(d)
        dm = max(dm,norm(d[i]))
    end
    return dm
end
function eval_mask(μv,huv)  
    # Extract active inequality constraints
    # active set mask
    mask = Diagonal(zeros(length(huv)))
    for i = 1:length(huv)
        mask[i,i] = (μv[i] > 0 || huv[i] > 0)
    end
    mask
end
function iLQR(params,X,U,P,p,K,d,Xn,Un;atol=1e-3,max_iters = 250,verbose = true,ρ=1,ϕ=10)

    # first check the sizes of everything
    @assert length(X) == params.N
    @assert length(U) == params.N-1
    @assert length(X[1]) == params.nx
    @assert length(U[1]) == params.nu
    @assert length(ineq_con_u(params,U[1])) == params.ncu  # no constraint control
    @assert length(ineq_con_x(params,X[1])) == params.ncx  # no constraint state

    # keep track of trajectories for each iterate
    Xhist=[deepcopy(X) for i = 1:1000]

    # initial rollout
    N = params.N
    for i = 1:N-1
        X[i+1] = discrete_dynamics(params,X[i],U[i],i)
    end

    Xhist[1] .= X


    reg_min = 1e-8
    reg = reg_min
    reg_max = 1e2

    μ = [zeros(params.ncu) for i = 1:N-1]   # input constraints

    μx = [zeros(params.ncx) for i = 1:N]    # state constraints

    λ = zeros(params.nx)    # goal constraint

    # Inner loop to solve unconstrained problem (Riccati)
    for iter = 1:max_iters
        ΔJ = backward_pass!(params,X,U,P,p,d,K,reg,μ,μx,ρ,λ)
        J, α = forward_pass!(params,X,U,K,d,ΔJ,Xn,Un,μ,μx,ρ,λ)

        Xhist[iter + 1] .= X

        # reg = update_reg(reg,reg_min,reg_max,α)
        dmax = calc_max_d(d)    
        if verbose
            if rem(iter-1,10)==0
                @printf "iter     J           ΔJ        |d|         α        reg         ρ\n"
                @printf "---------------------------------------------------------------------\n"
            end
            @printf("%3d   %10.3e  %9.2e  %9.2e  %6.4f   %9.2e   %9.2e\n",
              iter, J, ΔJ, dmax, α, reg, ρ)
        end

        # Iterate until inner loop converges (feasible linesearch and zero feedforward)
        # Then do outer update
        # if (α > 0) & (dmax<atol) 
            println("outer loop")
            # update multipliers and check constraint violation, max of all
            convio = 0  

            # control constraints (inequality)
            for k = 1:N-1
                huv = ineq_con_u(params,U[k])
                mask = eval_mask(μ[k],huv)
                # update dual
                μ[k] = max.(0,μ[k] + ρ*mask*huv)
                convio = max(convio,norm(huv + abs.(huv),Inf))
            end

#             # state constraints (inequality)
#             for k = 1:N
#                 hxv = ineq_con_x(params,X[k])
#                 mask = eval_mask(μx[k],hxv)

#                 # update dual
#                 μx[k] = max.(0,μx[k] + ρ*mask*hxv)
#                 convio = max(convio,norm(hxv + abs.(hxv),Inf))
#             end

            # # goal constraint (equality)
            # hxv = X[N] - params.Xref[N]
            # λ .+= ρ*hxv
            # convio = max(convio, norm(hxv,Inf))

            @show convio
            if convio <1e-4  # if terminal condition with contraint violation
                @info "success!"
                return Xhist[1:(iter + 1)];
            end

            ρ *= ϕ  # update penalty
        # end
    end
    error("iLQR failed")
end
#----------------------ALTRO DONE --------------------------

iLQR (generic function with 1 method)

In [121]:
# =============================
# Example problem formulation and solve
# =============================

# ---------------------THIS IS WHAT YOU NEED TO INPUT--------
function dynamics(p::NamedTuple,x,u,k)
    # dynamis for a cart pole
    mc = p.mc
    mp = p.mp
    l = p.l
    g = p.g

    q = x[1:2]
    qd = x[3:4]

    s = sin(q[2])
    c = cos(q[2])

    H = [mc+mp mp*l*c; mp*l*c mp*l^2]
    C = [0 -mp*qd[2]*l*s; 0 0]
    G = [0, mp*g*l*s]
    B = [1, 0]

    qdd = -H\(C*qd + G - B*u[1])
    return [qd; qdd]
end
function discrete_dynamics(p::NamedTuple,x,u,k)
    # RK4
    k1 = p.dt*dynamics(p,x,        u, k)
    k2 = p.dt*dynamics(p,x + k1/2, u, k)
    k3 = p.dt*dynamics(p,x + k2/2, u, k)
    k4 = p.dt*dynamics(p,x + k3, u, k)
    x + (1/6)*(k1 + 2*k2 + 2*k3 + k4)
end
function ineq_con_x(p,x)
    [x-p.x_max;-x + p.x_min]
end
function ineq_con_u(p,u)
    [u-p.u_max;-u + p.u_min]
end
function ineq_con_u_jac(params,u)
    FD.jacobian(_u -> ineq_con_u(params,_u), u)
end
function ineq_con_x_jac(p,x)
    FD.jacobian(_x -> ineq_con_x(p,_x),x)
end

ineq_con_x_jac (generic function with 1 method)

In [ ]:
# Let's solve
let
    nx = 4
    nu = 1
    N = 50
    dt = 0.1
    x0 = [0,0,0,0.]
    xg = [0,pi,0,0]
    Xref = [deepcopy(xg) for i = 1:N]
    Uref = [zeros(nu) for i = 1:N-1]
    Q = 1e-2*Diagonal([1,1,1,1.0])
    R = 1e-1*Diagonal([1.0])
    Qf = 1*Diagonal([1,1,1,1.0])

    u_min = -20*ones(nu)
    u_max =  20*ones(nu)

    # state is x y v θ
    x_min = -20*ones(nx)
    x_max =  20*ones(nx)

    ncx = 2*nx
    ncu = 2*nu

    params = (
        nx = nx,
        nu = nu,
        ncx = ncx,
        ncu = ncu,
        N = N,
        Q = Q,
        R = R,
        Qf = Qf,
        u_min = u_min,
        u_max = u_max,
        x_min = x_min,
        x_max = x_max,
        Xref = Xref,
        Uref = Uref,
        dt = dt,
        mc = 1.0,
        mp = 0.2,
        l = 0.5,
        g = 9.81,
    );

    # previous iterate
    X = [deepcopy(x0) for i = 1:N]
    U = [.01*randn(nu) for i = 1:N-1]

    # new iterate
    Xn = deepcopy(X)
    Un = deepcopy(U)


    P = [zeros(nx,nx) for i = 1:N]   # cost to go quadratic term
    p = [zeros(nx) for i = 1:N]      # cost to go linear term
    d = [zeros(nu) for i = 1:N-1]    # feedforward control
    K = [zeros(nu,nx) for i = 1:N-1] # feedback gain
    Xhist = iLQR(params,X,U,P,p,K,d,Xn,Un;atol=1e-1,max_iters = 3000,verbose = true,ρ = 1e0, ϕ = 10.0);
end

In [148]:
# =============================
# Example problem formulation and solve
# =============================

# ---------------------THIS IS WHAT YOU NEED TO INPUT--------
function discrete_dynamics(p::NamedTuple,x,u,k)
    # RK4
    A = [1.0  0.0  0.1  0.0;
         0.0  1.0  0.0  0.1;
         0.0  0.0  1.0  0.0;
         0.0  0.0  0.0  1.0]
    B = [0.005  0.0;
         0.0    0.005;
         0.1    0.0;
         0.0    0.1]
    return A*x + B*u
end
function ineq_con_x(p,x)
    [x-p.x_max;-x + p.x_min]
end
function ineq_con_u(p,u)
    [u-p.u_max;-u + p.u_min]
end
function ineq_con_u_jac(params,u)
    FD.jacobian(_u -> ineq_con_u(params,_u), u)
end
function ineq_con_x_jac(p,x)
    FD.jacobian(_x -> ineq_con_x(p,_x),x)
end

ineq_con_x_jac (generic function with 1 method)

In [157]:
# Let's solve

nx = 4
nu = 2
N = 51
dt = 0.1
t_vec = dt*(0:N-1)
x0 = [5,7,2,-1.4]
xg = [0,0,0,0]
Xref = [deepcopy(xg) for i = 1:N]
Uref = [zeros(nu) for i = 1:N-1]
Q = 1e-2*Diagonal([1,1,1,1.0])
R = 1e-1*Diagonal([1.0,1.0])
Qf = 100e-2*Q

u_min = -2*ones(nu)
u_max =  2*ones(nu)

# state is x y v θ
x_min = -20*ones(nx)
x_max =  20*ones(nx)

ncx = 2*nx
ncu = 2*nu

params = (
    nx = nx,
    nu = nu,
    ncx = ncx,
    ncu = ncu,
    N = N,
    Q = Q,
    R = R,
    Qf = Qf,
    u_min = u_min,
    u_max = u_max,
    x_min = x_min,
    x_max = x_max,
    Xref = Xref,
    Uref = Uref,
    dt = dt,
    mc = 1.0,
    mp = 0.2,
    l = 0.5,
    g = 9.81,
);

# previous iterate
X = [deepcopy(x0) for i = 1:N]
U = [.01*randn(nu) for i = 1:N-1]

# new iterate
Xn = deepcopy(X)
Un = deepcopy(U)

P = [zeros(nx,nx) for i = 1:N]   # cost to go quadratic term
p = [zeros(nx) for i = 1:N]      # cost to go linear term
d = [zeros(nu) for i = 1:N-1]    # feedforward control
K = [zeros(nu,nx) for i = 1:N-1] # feedback gain
Xhist = iLQR(params,X,U,P,p,K,d,Xn,Un;atol=1e-1,max_iters = 3000,verbose = true,ρ = 1e0, ϕ = 10.0);
function mat_from_vec(X::Vector{Vector{Float64}})::Matrix
    # convert a vector of vectors to a matrix 
    Xm = hcat(X...)
    return Xm 
end
Xsim_m = mat_from_vec(Xn)
Usim_m = mat_from_vec(Un)
# using Plots
# display(plot(t_vec,Xsim_m',label = ["x₁" "x₂" "ẋ₁" "ẋ₂"],
#              title = "State History",
#              xlabel = "time (s)", ylabel = "x"))
# display(plot(t_vec[1:end-1],Usim_m',label = ["u₁" "u₂"],
#              title = "Input History",
#              xlabel = "time (s)", ylabel = "u"))

solve riccati
line search
iter     J           ΔJ        |d|         α        reg         ρ
---------------------------------------------------------------------
  1    3.222e+01   4.20e+01   3.71e+00  1.0000    1.00e-08    1.00e+00
outer loop
convio = 2.2286639925325735
solve riccati
line search
  2    3.044e+01   3.59e+01   1.21e+00  1.0000    1.00e-08    1.00e+01
outer loop
convio = 0.4849220344384717
solve riccati
line search
  3    1.747e+01   9.82e+00   2.66e-01  1.0000    1.00e-08    1.00e+02
outer loop
convio = 0.0


┌ Info: success!
└ @ Main In[154]:364


2×50 Matrix{Float64}:
 -1.9999    -1.99992   -1.99994   …  0.0385146  0.0267781  0.0138962
 -0.932717  -0.815799  -0.706475     0.03951    0.0256233  0.0124177